In [88]:
import pandas as pd
import numpy as np
from pyswarm import pso
from shapely import Point, LineString
import geopandas as gpd
import matplotlib.pyplot as plt

In [89]:
data_path = '1m_precision_staiLine_11.csv'
swath_path = '../Scratch/test_Load_Shp/test_shps/swath_group_1.shp'
raw_data = pd.read_csv(data_path)
real_swath = gpd.read_file(swath_path)

In [90]:
# 处理 raw_data
# raw_data['height'] = raw_data['height'].apply(ast.literal_eval)
# raw_data['aspect'] = raw_data['aspect'].apply(ast.literal_eval)
# raw_data['slope'] = raw_data['slope'].apply(ast.literal_eval)
data_array = raw_data.to_numpy()
data_array

In [91]:
# 测试随机取出 
random_sample = np.random.choice(data_array.shape[0], 20, replace=False)
random_sample = data_array[random_sample]
random_sample

In [97]:
def fitness_function(lambdas, origin_x, origin_y, height, aspect, slope, labels):
    # 将 aspect 转换为 弧度值
    aspect = np.deg2rad(aspect)
    x_off = lambdas[0] * np.cos(aspect) * slope
    y_off = lambdas[1] * np.cos(aspect) * slope
    
    new_x = origin_x + x_off
    new_y = origin_y + y_off
    
    # optimize
    distances = []
    for temp_x, temp_y, label in zip(new_x, new_y, labels):
        temp_point = Point(temp_x, temp_y)
        temp_real_swath = real_swath.geometry.iloc[label]
        distances.append(temp_point.distance(temp_real_swath))
    return np.mean(distances)

In [99]:
lower_bound, upper_bound = [-1, -1], [1, 1]
results = []
EPOCH = 2
SAMPLE_BATCH=200

for i in range(EPOCH):
    print('---------------Epoch: {}---------------'.format(i))
    # random choice sample data
    random_sample = np.random.choice(data_array.shape[0], SAMPLE_BATCH, replace=True)
    random_sample = data_array[random_sample]
    # read data from random sample
    batch_origin_x = random_sample[:, 0]
    batch_origin_y = random_sample[:, 1]
    batch_height = random_sample[:, 2]
    batch_aspect = random_sample[:, 3]
    batch_slope = random_sample[:, 4]
    batch_labels = random_sample[:, 5].astype(np.int32)
    
    args = (batch_origin_x, batch_origin_y, batch_height, batch_aspect, batch_slope, batch_labels)
    xopt, fopt = pso(fitness_function, lower_bound, upper_bound, args=args)
    # showing temp result
    print('Optimized offs: ', xopt)
    print('Fopt: ', fopt)
    results.append((xopt, fopt))

In [100]:
new_results = np.array(results)
new_results

In [101]:
plt.plot(new_results[:, 1])